In [2]:
import dynet_config
dynet_config.set(random_seed=0)
import dynet as dy
import numpy as np

### Idea for the training corpus: 

Randomly generate sets of vectors and try to learn their average.

Generate 100000 (_number_of_examples_) samples of 2 (_number_of_vecors_) vectors that have a length of 10 (_vector_length_), then compute (as labels) their average. The learned weights for each vector should then be each 0.5.

In [3]:
# Generate corpus
number_of_examples = 100000
vector_length = 10
number_of_vecors = 2

X = np.random.rand(
    number_of_examples,
    number_of_vecors,
    vector_length
)

y = np.array([np.mean(x, axis=0) for x in X])

In [4]:
# I have no idea why exactly the computational graph has to be renewed.
dy.renew_cg();

# The model keeps track of all the variables that we want to update.
model = dy.ParameterCollection()
w = model.add_parameters(0, init=1.0, name="weights")

# Optimizer
trainer = dy.SimpleSGDTrainer(model, learning_rate=0.000003)

for epoch in range(100):
    cum_loss = []
    for i in range(number_of_examples):
        # Wrap everything in a tensor for dynet, because it needs
        # everything to be in this format.
        X_ = dy.inputTensor(X[i])
        y_ = dy.inputTensor(y[i])
        
        # Do some calculations:
        # (this does not make sense so far, but I wasn't able to recreate the
        # example I did with numpy in dynet. The dimensions didn't matched...)
        v = dy.cmult(X_, w)
        
        # Compute loss
        loss = dy.squared_distance(v, y_)
        
        # Save the loss so that we can see if each epoch is getting better
        cum_loss.append(loss.value())
        
        # Backpropagation
        loss.backward()
        trainer.update()
        
        # ???
        dy.renew_cg();
    
    print(sum(cum_loss))
    print(w.value())

ValueError: CwiseMultiply: For each dimension, the dim size needs to match or equal 1: [{2,10} {0}]